In [1]:
import sys # for automation and parallelisation
manual, scenario = (True, 'base') if 'ipykernel' in sys.argv[0] else (False, sys.argv[1])

In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
from quetzal.model import stepmodel
from quetzal.engine import engine, connectivity
from quetzal.io import excel

# Preparation of the transport network.
## Saves access and egress links for road network
## Needs all networks.


In [3]:
input_path = '../input_static/'
output_path = '../output/'
model_path = '../model/' + scenario + '/'

In [4]:
# Load scenario parameters
params = excel.read_var(file='../input/parameters.xls', scenario=scenario)

In [5]:
sm = stepmodel.read_json(model_path + 'de_zones')
road = stepmodel.read_json(input_path + 'de_road_network')

In [6]:
sm.zones.index = sm.zones.index.astype(str)

In [7]:
len(sm.zones)

4602

## Add access and egress links from zone centroids

In [8]:
# Compute controids
sm.centroids = gpd.GeoDataFrame(sm.zones[['FID', 'NUTS_ID']],
                                geometry=gpd.points_from_xy(sm.zones['lat'], sm.zones['lon'], crs=sm.epsg),
                                crs=sm.epsg)

In [9]:
# Compute road access and egress links
sm.zone_to_road = engine.ntlegs_from_centroids_and_nodes(
    sm.centroids,
    road.road_nodes,
    short_leg_speed=params['pt_access']['speed_walk'],
    long_leg_speed=params['pt_access']['speed_bicycle'],
    threshold=params['pt_access']['catchment_radius_walk'],
    n_neighbors=params['access-egress_links']['n_road'],
    coordinates_unit=sm.coordinates_unit)
sm.zone_to_road = sm.zone_to_road.loc[
    sm.zone_to_road['distance']<=params['access-egress_links']['road_max_dist']]
sm.zone_to_road.shape

(9186, 11)

### Parametrise access and egress links

In [10]:
# Road - centroid connectors
sm.zone_to_road['distance'] = 0
sm.zone_to_road['time'] = 0 # comes later
sm.zone_to_road.sample(n=2)

,a,b,rank,distance,geometry,direction,speed_factor,short_leg_speed,long_leg_speed,speed,time
4516,160690062,295550814,0,0,"LINESTRING (10.540413 50.398372, 10.5415946 50...",access,0.728243,5,15,5.000000,0
3738,120645404,10121713056,0,0,"LINESTRING (14.5044046 52.5765934, 14.5028988 ...",access,1.243592,5,15,6.217962,0


## Save model

In [11]:
# Drop unneccessary columns
cols = ['speed_factor', 'short_leg_speed', 'long_leg_speed', 'rank']
sm.zone_to_road.drop(cols, axis=1, inplace=True, errors='ignore')
#sm.road_to_transit.drop(cols, axis=1, inplace=True, errors='ignore')

In [12]:
# Make tables lighter
cols = ['distance', 'speed', 'time']
sm.zone_to_road[cols] = sm.zone_to_road[cols].astype(int)
#sm.road_to_transit[cols] = sm.road_to_transit[cols].astype(int)

In [13]:
sm.to_json(model_path + 'de_road_access_egress',
           only_attributes=['centroids', 'zone_to_road'], #, 'road_to_transit'],
           encoding='utf-8')

to_hdf(overwriting): 100%|█████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  5.09it/s]
